In [1]:
import pandas as pd
import datatable as dt

import numpy as np
import sys

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [ ]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote
from urllib.error import HTTPError
from zipfile import ZipFile

CHUNK_SIZE = 40960 
DATASET_MAPPING = 'riiid-test-answer-prediction:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F21651%2F1595136%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20201229%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20201229T155735Z%26X-Goog-Expires%3D259199%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D73692e0b98f75319be735352451b6e5fae4b365873e2e43a882bb3d6dc3698c02d4350ca9ac31c4df94069440668c28ac0f593f8d53513ea8e0be93d51eeb129d52a15fd7dca03a91ab71bfd99019b303e89f19cbb9d0949281715bcca799f22780ef780bd8d037a0ea7b6fb5d206519fe4c16a7751bf843d2f7e71b71f9f54fd05695beb23317d6a1e1dc247138fc661770f2606e98b3d425593ecc93eb982c769f92650c2bf53310ca5d917313375c39d4de6af1cf1e51602ae1824caa8ecd5ff9ad2917c628b54845ee99ee21effc9fd5a9b766749c1b70e2aeddea983d5e1339a07fb2724cab344e2b540576f88bb6c33bbc632e2f53a6099b004e6750e8,python-datatable:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F912556%2F1741730%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20201229%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20201229T155735Z%26X-Goog-Expires%3D259199%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6bed28b392cd5a98ac496b9c924fe13d05eeac7198512a5350c0909e962cac7be5c4a1fcd82ad7d0e4ab960f6e064a3cfc962ae0df1b6770abdcd40dde714a184f638f6c473ee37fc6b84a878b850331c41c059a7b763bd59d08ba975637621a676a17acb56b99a6258749c46f7d0343c02c6f86a270530651666ac1ef437a6877b9ede09fd80484fec791348c1c312112d36d7df2b1382573ce10090cb0fbed5330c7d642af90900b1eb6a121c6a706c6b1864b9e74976f224e9e301b4f52a68c1f2710d696a742f1bfe1197f59fc5b6d45c2f0302aba4fb85f5fff566128755a8dfcd37f062132a4f95f4178b325eaefea96291e306c494b23648908dc2e07'
KAGGLE_INPUT_PATH='/home/kaggle/input'
KAGGLE_INPUT_SYMLINK='/kaggle'

os.makedirs(KAGGLE_INPUT_PATH, 777)
os.symlink(KAGGLE_INPUT_PATH, os.path.join('..', 'input'), target_is_directory=True)
os.makedirs(KAGGLE_INPUT_SYMLINK)
os.symlink(KAGGLE_INPUT_PATH, os.path.join(KAGGLE_INPUT_SYMLINK, 'input'), target_is_directory=True)

for dataset_mapping in DATASET_MAPPING.split(','):
    directory, download_url_encoded = dataset_mapping.split(':')
    download_url = unquote(download_url_encoded)
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as zipfileres, NamedTemporaryFile() as tfile:
            total_length = zipfileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes zipped')
            dl = 0
            data = zipfileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = zipfileres.read(CHUNK_SIZE)
            print(f'\nUnzipping {directory}')
            with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue
print('Dataset import complete.')


# **Generating features**

In [2]:
train_data = dt.fread("/input/riiid-test-answer-prediction/train.csv").to_pandas()

def gen_q_data():
    
    idx = train_data.content_type_id.where(train_data.content_type_id == True).dropna().index.tolist() #indexes to drop
    
    group = train_data.drop(index=idx).groupby("content_id") #Filter unanswered questions
    group = group[["user_answer", "answered_correctly"]].aggregate(["value_counts"])

    cols = ["question_id", "0", "1", "2", "3", "que_false_per", "que_correct_per", "options_number", "correctness_number"]
    questions_info = pd.DataFrame(columns=cols)


    for a, b in group.groupby(level=0): #For every question values

        """ Restructure the input dataframe """
        vals = b.reset_index(level=[0,1]).copy()
        vals.columns = vals.columns.droplevel()
        vals = vals[["value_counts"]]
        vals.columns = ["answer", "correctness"]

        """ Get the answers of this question, and fill in the blanks for non-existant values """
        answers = vals[["answer"]].values.flatten().tolist()
        answer_number = len(answers)
        answers = answers + [0]*(4-len(answers))

        """ How many correct and how many wrong, and fill in the blanks for non-existant values """
        correct = np.nan_to_num(vals[["correctness"]].values[:2]).flatten().tolist()
        correct_number = len(correct)
        if len(correct) == 1: correct = [0] + correct

        #Merge them all in one list and append them to the dataframe
        row = [a] + answers + correct + [answer_number] + [correct_number]
        questions_info = questions_info.append(pd.Series(row, index=cols), ignore_index=True)


    """ Fill in nans and re-index """
    questions_info = questions_info.fillna(0)
    questions_info = questions_info.set_index("question_id")
    questions_info.index = questions_info.index.astype(int)

    """ Transform the values into percentages """
    total = questions_info["0"]+questions_info["1"]+questions_info["2"]+questions_info["3"]

    questions_info["0"] = questions_info["0"] / total
    questions_info["1"] = questions_info["1"] / total
    questions_info["2"] = questions_info["2"] / total
    questions_info["3"] = questions_info["3"] / total

    questions_info["que_false_per"] = questions_info["que_false_per"] / total
    questions_info["que_correct_per"] = questions_info["que_correct_per"] / total
    
    questions_info = questions_info.astype({'0': 'float16', 
                       '1': 'float16', 
                       '2': 'float16', 
                       '3': 'float16', 
                       'que_false_per': 'float16', 
                       'que_correct_per':'float16', 
                       'options_number':'int8', 
                       'correctness_number':'int8'})

    return questions_info

In [3]:
def questions_process(que_info, que_data):
    
    tag = que_data["tags"].str.split(" ", n = 10, expand = True) 
    tag.columns = ['tags1','tags2','tags3','tags4','tags5','tags6']

    que_data =  pd.concat([que_data,tag],axis=1).drop(['tags'],axis=1)
    que_data['tags1'] = pd.to_numeric(que_data['tags1'], errors='coerce',downcast='integer').fillna(-1)
    que_data['tags2'] = pd.to_numeric(que_data['tags2'], errors='coerce',downcast='integer').fillna(-1)
    que_data['tags3'] = pd.to_numeric(que_data['tags3'], errors='coerce',downcast='integer').fillna(-1)
    que_data['tags4'] = pd.to_numeric(que_data['tags4'], errors='coerce',downcast='integer').fillna(-1)
    que_data['tags5'] = pd.to_numeric(que_data['tags5'], errors='coerce',downcast='integer').fillna(-1)
    que_data['tags6'] = pd.to_numeric(que_data['tags6'], errors='coerce',downcast='integer').fillna(-1)
    
    que_data = que_data.merge(que_info, left_on="question_id", right_on="question_id", how="right") #Merge all questions data into one df
    
    
    que_data.tags1 = que_data.tags1.astype("int16")
    que_data.tags2 = que_data.tags2.astype("int16")
    que_data.tags3 = que_data.tags3.astype("int16")
    que_data.tags4 = que_data.tags4.astype("int16")
    que_data.tags5 = que_data.tags5.astype("int16")
    que_data.tags6 = que_data.tags6.astype("int16")
    que_data.question_id = que_data.question_id.astype("int16")
    que_data.bundle_id = que_data.bundle_id.astype("int16")
    que_data.correct_answer = que_data.correct_answer.astype("int8")
    que_data.part = que_data.part.astype("int8")
    
    return que_data

In [4]:
import gc

gc.collect()

train_data.task_container_id = train_data.task_container_id.astype("int16")
train_data.user_answer = train_data.user_answer.astype("int8")
train_data.answered_correctly = train_data.answered_correctly.astype("int8")

train_data.prior_question_elapsed_time = train_data.prior_question_elapsed_time.fillna(0)
train_data.prior_question_elapsed_time = train_data.prior_question_elapsed_time.astype("uint32")

train_data.prior_question_had_explanation = train_data.prior_question_had_explanation.fillna(False)
train_data.prior_question_had_explanation = train_data.prior_question_had_explanation.astype("int8")

gc.collect()


questions_info = gen_q_data()

folder_path = '/input/riiid-test-answer-prediction/'
que_csv =   folder_path + 'questions.csv'

que_data = pd.read_csv(que_csv)
que_data = questions_process(questions_info, que_data)

del que_data["question_id"] 

In [5]:
que_data.to_pickle("/home/dfs/que_data.pickle")
del que_data

In [6]:
a = train_data.groupby("user_id")["answered_correctly"].count()

prob_users = train_data[(train_data.timestamp == 0) & (train_data.prior_question_elapsed_time != 0)].user_id.unique().tolist()
prob_users.extend([171953341,1958652827])  #These two users only answer to one bundle
prob_users.extend(a[a == 1].index.tolist())
prob_users = list(set(prob_users))

del a

In [7]:
train_data = train_data[~train_data.user_id.isin(prob_users)]

In [8]:
train_data["task_con"] = (train_data.timestamp != train_data.timestamp.shift()).cumsum()

In [9]:
train_data["rolling_mean"] = train_data.groupby("user_id")["answered_correctly"].apply(lambda x: x.ewm(span=5, adjust=False).mean())
train_data["rolling_mean"] = train_data.groupby("user_id")["rolling_mean"].shift(1).fillna(0).astype("float32")

In [10]:
#Feature lag
train_data["rolling_mean"] = train_data.groupby(["user_id","task_con"])["rolling_mean"].transform("first")

In [11]:
gc.collect()

44

In [12]:
"""All of the created features are rolling features"""

#The inverse of content_type_id, to be user for summing
train_data["ques"] = train_data["content_type_id"] == False

#Ques_right, a row that specifies if question is right or wrong, it sets value 0 for a lecture
train_data["ques_right"] = train_data["answered_correctly"]
train_data.loc[train_data["ques_right"] ==  -1, "ques_right"] = 0


#How many correct answer per user
train_data["correct_count"] = train_data.groupby("user_id")["ques_right"].cumsum()
train_data["correct_count"] = train_data.groupby("user_id")["correct_count"].shift(1) #Avoid data leak

#How many question this user answered
train_data["que_count_user"] = train_data.groupby("user_id")["ques"].cumsum()
train_data["que_count_user"] = train_data.groupby("user_id")["que_count_user"].shift(1) #Avoid data leak

#How many time this user repeated this specific question
train_data["question_repeated"] = train_data.groupby(["user_id","content_id"])["ques"].cumsum()

#User current accuracy
train_data["user_mean"] = (train_data["correct_count"] / train_data["que_count_user"]).fillna(0.55)

In [13]:
#Fillna caused by shift
train_data.correct_count = train_data.correct_count.fillna(0)
train_data.que_count_user = train_data.que_count_user.fillna(0)

train_data.question_repeated = train_data.question_repeated.astype("int8")
train_data.que_count_user = train_data.que_count_user.astype("int16")
train_data.correct_count = train_data.correct_count.astype("int16")
train_data.user_mean = train_data.user_mean.astype("float32")


#Fix lag
train_data["correct_count"] = train_data.groupby(["user_id","task_con"])["correct_count"].transform("first")
train_data["que_count_user"] = train_data.groupby(["user_id","task_con"])["que_count_user"].transform("first")
train_data["user_mean"] = train_data.groupby(["user_id","task_con"])["user_mean"].transform("first")


del train_data["ques_right"]
del train_data["ques"]

In [14]:
"""Creating a row with the last lecture watched by every student"""
train_data["last_lecture"] = train_data["content_id"].where(train_data["content_type_id"])
train_data["last_lecture"] = train_data.groupby("user_id")["last_lecture"].apply(lambda x: x.fillna(method="ffill"))
train_data.last_lecture = train_data.last_lecture.fillna(0)

train_data.last_lecture = train_data.last_lecture.astype("uint16")

In [15]:
del train_data["row_id"]
gc.collect()

22

In [16]:
train_data.to_pickle("/home/dfs/data.pickle")
del train_data

In [17]:
%reset -f

In [18]:
import pandas as pd

#supress warnings
import warnings
warnings.filterwarnings("ignore")

import datatable as dt
import numpy as np
import sys
import gc


In [19]:
lec_data = dt.fread("/input/riiid-test-answer-prediction/lectures.csv").to_pandas()
lec_dict = lec_data[["lecture_id", "tag"]].set_index("lecture_id").to_dict("index")

In [20]:
folder_path = '/input/riiid-test-answer-prediction/'
que_csv =   folder_path + 'questions.csv'

que_data = pd.read_csv(que_csv)

tag = que_data["tags"].str.split(" ", n = 10, expand = True) 
tag.columns = ['tags1','tags2','tags3','tags4','tags5','tags6']

que_data =  pd.concat([que_data,tag],axis=1).drop(['tags'],axis=1)
que_data['tags1'] = pd.to_numeric(que_data['tags1'], errors='coerce',downcast='integer').fillna(-1)
que_data['tags2'] = pd.to_numeric(que_data['tags2'], errors='coerce',downcast='integer').fillna(-1)
que_data['tags3'] = pd.to_numeric(que_data['tags3'], errors='coerce',downcast='integer').fillna(-1)
que_data['tags4'] = pd.to_numeric(que_data['tags4'], errors='coerce',downcast='integer').fillna(-1)
que_data['tags5'] = pd.to_numeric(que_data['tags5'], errors='coerce',downcast='integer').fillna(-1)
que_data['tags6'] = pd.to_numeric(que_data['tags6'], errors='coerce',downcast='integer').fillna(-1)

In [21]:
train_data = dt.fread("/input/riiid-test-answer-prediction/train.csv")

In [22]:
del train_data["prior_question_elapsed_time"]
del train_data["prior_question_had_explanation"]
del train_data["task_container_id"]
del train_data["timestamp"]

In [23]:
train_data = train_data.to_numpy()

In [24]:
cols = ["row_id","user_id","content_id","content_type_id","user_answer","answered_correctly"]
col2num = {cols[i]:i for i in range(len(cols))}

In [25]:
from tqdm import tqdm

In [26]:
features = np.zeros((train_data.shape[0], 8),dtype=np.int16)

que_data.set_index("question_id", drop=True, inplace=True)
que_data = que_data.to_dict("index")

In [27]:
lecs_watched_tags = {}

In [28]:
for i in tqdm(range(train_data.shape[0])):
    
    user_id = train_data[i,col2num['user_id']]
    content_id = train_data[i,col2num['content_id']]
    content_type_id = train_data[i,col2num['content_type_id']]
    correct = train_data[i,col2num['answered_correctly']]
    
    if content_type_id:
        if lecs_watched_tags.get(str(user_id), -1) == -1:  #If user's first lecture
            lecs_watched_tags[str(user_id)] = {}

        lec_tag  = lec_dict[content_id]['tag']
        lecs_watched_tags[str(user_id)][str(lec_tag)] = 1
      
    else:
        
        question = que_data[content_id]

        for j in range(6):
            
            tag = int(question["tags" + str(j+1)])
            if tag == -1:
                features[i, j] = -1
            else:
                features[i, j] = lecs_watched_tags.get(str(user_id), {}).get(str(tag), 0)
        

100%|██████████| 101230332/101230332 [35:59<00:00, 46879.31it/s] 


In [29]:
del train_data
gc.collect()

36

In [30]:
new_cols = ["watched_1","watched_2","watched_3","watched_4","watched_5","watched_6","tag_que_count", "tag_que_correct"]
features = pd.DataFrame(features, columns=new_cols)

In [31]:
features = features.astype({"watched_1":"int8", "watched_2":"int8", "watched_3":"int8", 
                 "watched_4":"int8", "watched_5":"int8", "watched_6":"int8"})

In [32]:
gc.collect()

66

In [33]:
del features["tag_que_count"]
del features["tag_que_correct"]

In [34]:
features.to_pickle("/home/dfs/features.pickle")
del features

In [35]:
%reset -f

In [17]:
import sys
import datatable as dt
import numpy as np

In [18]:
import os
import glob
import time
from os import listdir
from typing import Dict

import sklearn.preprocessing as preprocessing
from sklearn.metrics import roc_auc_score

import pandas as pd

from sys import getsizeof

#supress warnings
import warnings
warnings.filterwarnings("ignore")

import time
import gc

features_1_path = ''
le = preprocessing.LabelEncoder()

In [19]:
from sklearn.cluster import KMeans

questions = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv')
questions = questions[~questions['tags'].isna()]
questions = questions.assign(tags=questions['tags'].str.split()).explode('tags')
questions = questions[['question_id', 'tags']]
df = pd.get_dummies(questions, columns=['tags'])

tmp_df = df.groupby('question_id').agg(['sum'])

km = KMeans(n_clusters=20,
            init='random',
            n_init=10,
            random_state=0)


X = tmp_df.iloc[:, 1:]
cluster_labels = km.fit_predict(X)

tmp_df['k'] = cluster_labels
tmp_df = tmp_df.reset_index()
tmp_df = tmp_df[['question_id', 'k']]
tmp_df.columns = tmp_df.columns.droplevel(1)
tmp_df.head(10)

question_cluster = tmp_df.set_index("question_id", drop=True)
del tmp_df

In [20]:
train = pd.read_pickle("/home/dfs/data.pickle")
que_data = pd.read_pickle("/home/dfs/que_data.pickle")
train.user_mean = train.user_mean.fillna(0)
train.rolling_mean = train.rolling_mean.fillna(0)

del train["user_answer"]
que_data["k"] = question_cluster

In [21]:
data = pd.read_pickle("/home/dfs/features.pickle")

train[["watched"]] = data[["watched_1","watched_2","watched_3","watched_4","watched_5","watched_6"]].replace(-1, 0).sum(axis=1)

gc.collect()

del data
#"tag_que_count", "tag_que_correct","tag_que_acc"
train.watched = train.watched.astype("int8")

In [22]:
train["timestamp"] = train["timestamp"]/8.64e+7
train["time_diff"] = train.groupby("user_id")["timestamp"].diff().fillna(0).astype("float32")

train.rolling_mean = train.rolling_mean.astype("float16")

In [23]:
train["sessions"] = train["time_diff"] > 0.083
train["sessions"] = train.groupby("user_id")["sessions"].cumsum()

train["session_count"] =1
train["session_count"] = train.groupby(["user_id","sessions"])["session_count"].cumsum()

In [24]:
train["mean_pause"] = ((train.time_diff > 0.083)*train.time_diff).replace(0,np.nan)
test = train[["mean_pause","user_id"]]
test = test.dropna()
test["kasem"] = (test.groupby("user_id").cumcount() + 1 )
test["cumsum"] = test.groupby("user_id")["mean_pause"].cumsum()
test["mean_pause"] = test["cumsum"]/test["kasem"]
del test["user_id"]
del test["kasem"]
del test["cumsum"]
del train["mean_pause"]
train = train.join(test)
train["mean_pause"] = train.groupby("user_id")["mean_pause"].fillna(method="ffill").fillna(0).astype("float16")

In [25]:
train["lecs_per"] = (train.groupby("user_id")["content_type_id"].cumsum()/(train.groupby("user_id")["content_type_id"].cumcount()+1))*100
train["lecs_per"] = train["lecs_per"].astype("float16")

In [26]:
train = train[train.content_type_id == False]
del train["content_type_id"]

train.answered_correctly = train.answered_correctly.astype("bool")

In [27]:
train.session_count = train.session_count.astype("int16") 
train.sessions = train.sessions.astype("int16") 

In [28]:
train = train.merge(que_data[["part", "k"]], how="left", right_index=True, left_on="content_id")
train.part = train.part.astype("int8")

In [29]:
#Maybe later replace them with user average for NaNs

train["elapsed_time"] = train.groupby(["user_id"])["prior_question_elapsed_time"].shift(-1)
train["elapsed_time"] = train.groupby(["user_id","task_con"])["elapsed_time"].transform("last").fillna(method="ffill").astype("uint32")

train["read_explanation"] = train.groupby(["user_id"])["prior_question_had_explanation"].shift(-1)
train["read_explanation"] = train.groupby(["user_id","task_con"])["read_explanation"].transform("last").fillna(method="ffill").astype("bool")

In [30]:
denom = train.groupby(["user_id","part"])["answered_correctly"].cumcount()+1

In [31]:
%%time
train["part_mean"] = train.groupby(["user_id","part"])["answered_correctly"].cumsum()/denom
train["part_mean"] = train.groupby(["user_id", "part"])["part_mean"].shift(1).fillna(0).astype("float16")
train["part_mean"] = train.groupby(["user_id", "task_con" ,"part"])["part_mean"].transform("first")

train["prior_question_had_explanation_u_part_avg"] = (train.groupby(["user_id", "part"])["read_explanation"].cumsum() / denom).astype("float32")
train["prior_question_elapsed_time_u_part_avg"]= (train.groupby(["user_id", "part"])["elapsed_time"].cumsum()/denom).astype("float32")

del denom
gc.collect()

CPU times: user 1min 15s, sys: 19.3 s, total: 1min 34s
Wall time: 1min 33s


44

In [32]:
#Data leakage avoidance
train["prior_question_had_explanation_u_part_avg"] = train.groupby(["user_id", "part"])["prior_question_had_explanation_u_part_avg"].shift(1).fillna(0)
train["prior_question_elapsed_time_u_part_avg"] = train.groupby(["user_id", "part"])["prior_question_elapsed_time_u_part_avg"].shift(1).fillna(0)

In [33]:
train["prior_question_had_explanation_u_part_avg"] = train.groupby(["user_id", "part", "task_con"])["prior_question_had_explanation_u_part_avg"].transform("first")
train["prior_question_elapsed_time_u_part_avg"] = train.groupby(["user_id", "part", "task_con"])["prior_question_elapsed_time_u_part_avg"].transform("first")

In [34]:
del train["part"]
gc.collect()

66

In [35]:
#Alternative way of rolling average since it is super slow
window = 5

tmp = train.groupby("user_id")["answered_correctly"].cumsum() - train.groupby("user_id")["answered_correctly"].cumsum().groupby(train["user_id"]).shift(window).fillna(0)
denom = (train.groupby("user_id")["answered_correctly"].cumcount() + 1).clip(upper=window)

train["rolling_mean_5"] = tmp / denom
del tmp
del denom

In [36]:
window = 10

tmp = train.groupby("user_id")["answered_correctly"].cumsum() - train.groupby("user_id")["answered_correctly"].cumsum().groupby(train["user_id"]).shift(window).fillna(0)
denom = (train.groupby("user_id")["answered_correctly"].cumcount() + 1).clip(upper=window)

train["rolling_mean_10"] = tmp / denom
del tmp
del denom

#===============================
window = 15

tmp = train.groupby("user_id")["answered_correctly"].cumsum() - train.groupby("user_id")["answered_correctly"].cumsum().groupby(train["user_id"]).shift(window).fillna(0)
denom = (train.groupby("user_id")["answered_correctly"].cumcount() + 1).clip(upper=window)

train["rolling_mean_15"] = tmp / denom
del tmp
del denom

In [37]:
train["rolling_mean_15"] = train.groupby("user_id")["rolling_mean_15"].shift(1).fillna(0).astype("float32")
train["rolling_mean_10"] = train.groupby("user_id")["rolling_mean_10"].shift(1).fillna(0).astype("float32")
train["rolling_mean_5"] = train.groupby("user_id")["rolling_mean_5"].shift(1).fillna(0).astype("float32")

In [38]:
train["rolling_mean_15"] = train.groupby(["user_id","task_con"])["rolling_mean_15"].transform("first")
train["rolling_mean_10"] = train.groupby(["user_id","task_con"])["rolling_mean_10"].transform("first")
train["rolling_mean_5"] = train.groupby(["user_id","task_con"])["rolling_mean_5"].transform("first")

In [39]:
#Might add same feature but for part

window = 5

tmp = train.groupby("user_id")["prior_question_had_explanation"].cumsum() - train.groupby("user_id")["prior_question_had_explanation"].cumsum().groupby(train["user_id"]).shift(window).fillna(0)
denom = (train.groupby("user_id")["prior_question_had_explanation"].cumcount() + 1).clip(upper=window)

train["rolling_mean_5_prior_question"] = tmp / denom
del tmp
del denom

In [40]:
train["ewm_mean_10"] = train.groupby("user_id")["answered_correctly"].apply(lambda x: x.ewm(span=5, adjust=False).mean())
train["ewm_mean_10"] = train.groupby("user_id")["ewm_mean_10"].shift(1).fillna(0).astype("float32")

train["ewm_mean_10"] = train.groupby(["user_id","task_con"])["ewm_mean_10"].transform("first")

gc.collect()

0

In [41]:
%%time
#Potential leak
train["correct_recency"] = train["timestamp"] * train["answered_correctly"]
train["correct_recency"] = train["correct_recency"].replace(0, np.nan)
train["correct_recency"] = train["timestamp"] - train.groupby("user_id")["correct_recency"].fillna(method="ffill").shift(1).fillna(0)

CPU times: user 1min 16s, sys: 4.93 s, total: 1min 21s
Wall time: 1min 20s


In [42]:
train["correct_recency"] = train.groupby(["user_id","task_con"])["correct_recency"].transform("first").clip(lower=0)

In [43]:
#THIS

train.k = train.k.fillna(0).astype("int8")
train["k_acc"] = (train.groupby(["user_id","k"])["answered_correctly"].cumsum())/(train.groupby(["user_id","k"]).cumcount() + 1)
train["k_acc"] = train.groupby(["user_id", "k"])["k_acc"].shift()

k_agc_acc = train.groupby("k")["answered_correctly"].mean()
train["k_acc"] = train.groupby(["user_id","task_con","k"])["k_acc"].transform("first")

In [44]:
train["el_avg"] = train.groupby("user_id")["prior_question_elapsed_time"].cumsum()/(train.groupby("user_id").cumcount()+1).astype("float32")

In [45]:
train["prior_question_had_explanation_ratio"] = train.groupby("user_id")["prior_question_had_explanation"].cumsum()/(train.groupby("user_id")["prior_question_had_explanation"].cumcount()+1) 
train.prior_question_had_explanation_ratio = train.prior_question_had_explanation_ratio.astype("float16") 

In [46]:
a = train.groupby("content_id")["answered_correctly"].agg(["count","mean"])

easy_questions = a[(a["mean"] > 0.8)]
easy_questions["easy_question"] = True

del easy_questions["mean"]
del easy_questions["count"]


hard_questions = a[(a["count"] > 100) & (a["mean"] < 0.4)]
hard_questions["hard_question"] = True

del hard_questions["mean"]
del hard_questions["count"]


train.loc[train.content_id.isin(hard_questions.index), "hard"] = True
train.hard = train.hard.fillna(False).astype("bool")

In [47]:
train["n_hard"] = train.groupby("user_id")["hard"].cumsum().astype("int16")

In [48]:

train["tmp"] = train["answered_correctly"]* train["hard"]
train["n_hard_correct"] = train.groupby("user_id")["tmp"].cumsum().astype("int16")

del train["tmp"]


train["hard_ratio"] = (train["n_hard_correct"]/train["n_hard"]).astype("float16")
train["hard_ratio"] = train.groupby("user_id")["hard_ratio"].shift().fillna(0).astype("float16")
del train["n_hard"]
del train["n_hard_correct"]
del train["hard"]

train.loc[train.content_id.isin(easy_questions.index), "easy"] = True
train.easy = train.easy.fillna(False).astype("bool")


train["n_easy"] = train.groupby("user_id")["easy"].cumsum().astype("int16")


train["tmp"] = train["answered_correctly"]* train["easy"]
train["n_easy_correct"] = train.groupby("user_id")["tmp"].cumsum().astype("int16")

del train["tmp"]

train["easy_ratio"] = (train["n_easy_correct"]/train["n_easy"]).astype("float16")
train["easy_ratio"] = train.groupby("user_id")["easy_ratio"].shift().fillna(0).astype("float16")
del train["n_easy"]
del train["n_easy_correct"]
del train["easy"]

In [49]:
train["easy_ratio"] = train.groupby(["user_id","task_con"])["easy_ratio"].transform("first")
train["hard_ratio"] = train.groupby(["user_id","task_con"])["hard_ratio"].transform("first")

In [50]:
train["time_diff1"] = train.groupby("user_id")["timestamp"].diff(2).fillna(0).astype("float32")
train["time_diff2"] = train.groupby("user_id")["timestamp"].diff(3).fillna(0).astype("float32")
train["time_diff3"] = train.groupby("user_id")["timestamp"].diff(4).fillna(0).astype("float32")
train["time_diff4"] = train.groupby("user_id")["timestamp"].diff(5).fillna(0).astype("float32")

In [51]:
train.k_acc = train.k_acc.fillna(0).astype("float16")

In [52]:
train["ts_diff_shifted"] = train.groupby("user_id")["time_diff"].shift(1).replace(0, np.nan)

In [53]:
train["ts_diff_shifted"] = train.groupby("user_id")["ts_diff_shifted"].fillna(method="ffill").fillna(0)*8.64e+7

In [54]:
#See if this need fix
train["ts_diff_shifted_2"] = train.groupby("user_id")["time_diff1"].shift(1).replace(0, np.nan)
train["ts_diff_shifted_2"] = train.groupby("user_id")["ts_diff_shifted_2"].fillna(method="ffill").fillna(0)*8.64e+7

In [55]:
train["task_cnt"] = train.groupby("task_con").cumcount()+1
train["task_size"] = train.groupby("task_con")["task_cnt"].transform("last")

In [56]:
train["wut"] = train["ts_diff_shifted"] - train["prior_question_elapsed_time"]*train["task_size"]

In [61]:
train.to_pickle("/home/dfs/data_2")

In [ ]:
!python generate-corr.py
!python create-corr-df.py
!python gen-valid-split.py

# **Model Training**

In [ ]:
vals = pd.read_pickle("/home/dfs/corr")

In [ ]:
train = train.join(vals, rsuffix="_l_")

In [ ]:
del train["content_id_l_"] 
del train["user_id_l_"]
del train["task_container_id_l_"]
del train["answered_correctly_l_"]
del train["user_id"]

In [ ]:
train["sum_corr"] = train["corr_1"] + train["corr_2"] + train["corr_3"] + train["corr_4"] + train["corr_5"]
train["gen_mean"] = train["user_mean"] - 0.657

In [ ]:
val_1 = pd.read_pickle("/home/dfs/cv1_valid.pickle")
val_2 = pd.read_pickle("/home/dfs/cv2_valid.pickle")

In [ ]:
val_idxs_1 = val_1.row_id
val_idxs_1 = val_idxs_1[np.isin(val_idxs_1, train.index)]

validation_1 = train.loc[val_idxs_1]

In [ ]:
val_idxs_2 = val_2.row_id
val_idxs_2 = val_idxs_2[np.isin(val_idxs_2, train.index)]

validation_2 = train.loc[val_idxs_2]

In [ ]:
train = train[~train.index.isin(val_1.row_id)]
train = train[~train.index.isin(val_2.row_id)]

In [ ]:
dropped_idxs = train.sample(frac=0.75).index
train.drop(index=dropped_idxs, inplace=True)

with open('dropped_idxs.npy', 'wb') as f:
    np.save(f, dropped_idxs.values)
    
del dropped_idxs
del val_idxs_1
del val_idxs_2

del val_1
del val_2

In [ ]:
containers_mean = train.groupby("task_container_id")["answered_correctly"].mean()

containers_mean = pd.DataFrame(data=containers_mean)
containers_mean.columns = ["container_mean"]

train = train.merge(containers_mean, how="left", left_on="task_container_id", right_index=True)
validation_1 = validation_1.merge(containers_mean, how="left", left_on="task_container_id", right_index=True)
validation_2 = validation_2.merge(containers_mean, how="left", left_on="task_container_id", right_index=True)

In [ ]:
train["opp_mean"] = 1 - train["user_mean"]
validation_1["opp_mean"] = 1 - validation_1["user_mean"]
validation_2["opp_mean"] = 1 - validation_2["user_mean"]

In [ ]:
que_count = train.groupby("content_id")["answered_correctly"].count()

que_count = pd.DataFrame(data=que_count)
que_count.columns = ["que_count"]

train = train.merge(que_count, how="left", left_on="content_id", right_index=True)
validation_1 = validation_1.merge(que_count, how="left", left_on="content_id", right_index=True)
validation_2 = validation_2.merge(que_count, how="left", left_on="content_id", right_index=True)

In [ ]:
que_data = pd.read_pickle("/home/dfs/que_data.pickle")

In [ ]:
#Add dtype for every feature
features = [
    'task_container_id', "ts_diff_shifted", "watched","ts_diff_shifted_2",
    'content_id', "k", "el_avg", "wut", "sum_corr", "gen_mean", "u_mean", "u_mean_opp",
    'prior_question_elapsed_time', "time_diff2", "rolling_mean_5", "rolling_mean_10", "rolling_mean_15", "prior_question_had_explanation_u_part_avg",
    'prior_question_had_explanation', "hard_ratio_opp", "easy_ratio_opp", "correct_recency", "prior_question_elapsed_time_u_part_avg", "ewm_mean_10", "rolling_mean_5_prior_question",
    'last_lecture', "part_mean", "opp_mean", "mean_pause", "timestamp", "prior_part_mean",
    "container_mean", "lecs_per", "hard_ratio", "easy_ratio", 'corr_1', 'corr_2', 'corr_3', 'corr_4','corr_5', 'top_que',
    'que_count_user', 'question_repeated', "rolling_mean","time_diff3", "time_diff4",
    'user_mean', "time_diff1", "time_diff", "sessions", "session_count", "prior_question_had_explanation_ratio"
] + que_data.columns.tolist()[:-1]

features.remove("options_number")
features.remove("correct_answer")
features.remove("tags6")
features.remove("tags5")
features.remove("tags4")


target_column = 'answered_correctly'

In [ ]:
train = train.merge(que_data, how="left", left_on="content_id", right_index=True)
validation_1 = validation_1.merge(que_data, how="left", left_on="content_id", right_index=True)
validation_2 = validation_2.merge(que_data, how="left", left_on="content_id", right_index=True)

gc.collect()

In [ ]:
train["u_mean"] = train["user_mean"] - train["que_correct_per"]
train["u_mean_opp"] = train["user_mean"] - train["que_false_per"]

validation_1["u_mean"] = validation_1["user_mean"] - validation_1["que_correct_per"]
validation_1["u_mean_opp"] = validation_1["user_mean"] - validation_1["que_false_per"]

validation_2["u_mean"] = validation_2["user_mean"] - validation_2["que_correct_per"]
validation_2["u_mean_opp"] = validation_2["user_mean"] - validation_2["que_false_per"]

In [ ]:
kkkk = pd.Series({1: 22166.159642501425,
 2: 18714.69673913695,
 3: 23620.317746179924,
 4: 23762.753651169547,
 5: 25094.620302855932,
 6: 32417.37918735745,
 7: 47444.16407400242})

In [ ]:
train = train.merge(pd.DataFrame(kkkk, columns=["prior_part_mean"]), how="left", left_on="part", right_index=True)

validation_1 = validation_1.merge(pd.DataFrame(kkkk, columns=["prior_part_mean"]), how="left", left_on="part", right_index=True)
validation_2 = validation_2.merge(pd.DataFrame(kkkk, columns=["prior_part_mean"]), how="left", left_on="part", right_index=True)


In [ ]:
train["hard_ratio_opp"] = (1 - train["hard_ratio"]).astype("float16")
train["easy_ratio_opp"] = (1 - train["easy_ratio"]).astype("float16")

validation_1["hard_ratio_opp"] = (1 - validation_1["hard_ratio"]).astype("float16")
validation_1["easy_ratio_opp"] = (1 - validation_1["easy_ratio"]).astype("float16")

validation_2["hard_ratio_opp"] = (1 - validation_2["hard_ratio"]).astype("float16")
validation_2["easy_ratio_opp"] = (1 - validation_2["easy_ratio"]).astype("float16")

In [ ]:
for i in train.columns:
    if not i in features + [target_column]:
        del train[i]
        del validation_1[i]
        del validation_2[i]

In [ ]:
import lightgbm as lgb

params = {'objective': 'binary', 'seed': 42, 'metric': 'auc', 'learning_rate': 0.032, 'max_bin': 800, 'num_leaves': 200, 'feature_fraction':0.5}

In [ ]:
train["el_avg"] = train.el_avg / 1000
validation_1["el_avg"] = validation_1.el_avg / 1000
validation_2["el_avg"] = validation_2.el_avg / 1000

In [ ]:
train.reset_index(drop=True, inplace=True)
validation_1.reset_index(drop=True, inplace=True)
validation_2.reset_index(drop=True, inplace=True)

In [ ]:
gc.collect()

tr_data = lgb.Dataset(train[features], label=train[target_column])
va_data1 = lgb.Dataset(validation_1[features], label=validation_1[target_column])
va_data2 = lgb.Dataset(validation_2[features], label=validation_2[target_column])

In [ ]:
import matplotlib.pyplot as plt

model = lgb.train(
    params, 
    tr_data, 
    num_boost_round=7000,
    valid_sets=[tr_data, va_data1, va_data2], 
    early_stopping_rounds=50,
    verbose_eval=50
)

In [ ]:
ax = lgb.plot_importance(model, importance_type='gain')
fig = ax.figure
fig.set_size_inches(10, 20)
fig.savefig('/home/foo.pdf')

In [ ]:
model.save_model('/home/lgb_classifier_final.txt', num_iteration=model.best_iteration) 

# **Creating needed dictionaries to re-create the features**

In [ ]:
%reset -f

In [ ]:
import pandas as pd
import datatable as dt
import numpy as np

train = dt.fread("/input/riiid-test-answer-prediction/train.csv").to_pandas()


In [ ]:
#Convert users into sequences
groups = train[['user_id', 'content_id', 'answered_correctly']].groupby('user_id').apply(lambda r: (
            r['content_id'].values,
            r['answered_correctly'].values))

In [ ]:
groups = groups.apply(lambda x: x[1][-140:].tolist())
groups = groups.to_dict()

In [ ]:
lec_data = pd.read_csv("/input/riiid-test-answer-prediction/lectures.csv")
lec_dict = lec_data[["lecture_id", "tag"]].set_index("lecture_id").tag.to_dict()

In [ ]:
que_data = pd.read_pickle("/home/dfs/que_data.pickle")
questions = que_data.to_dict("index")

In [ ]:
def gen_lec_data(train):
    
    train = train[train["content_type_id"] == True]
    
    train = train.groupby("user_id").apply(lambda x: x.content_id.tolist())
    train = pd.DataFrame(train)
    train.columns = ["CUM_CONCAT"]
    
    global lec_history
    lec_history = {}

    def f(row):

        global lec_history
        lec_history[str(row.name)] = {}

        for i in row.CUM_CONCAT:
            lec_tag  = lec_dict[i]
            lec_history[str(row.name)][str(lec_tag)] = 1

        return row

    train.apply(f, axis=1)
    
    return lec_history
    
watched_tags = gen_lec_data(train)

In [ ]:
#With lectures
timetable = train.groupby("user_id")[["user_id","timestamp"]].tail(5).groupby("user_id")["timestamp"].apply(lambda x: x.values)

In [ ]:
timetable_df = pd.DataFrame(columns=["first_timestamp","second_timestamp","third_timestamp","fourth_timestamp","fifth_timestamp"], index=timetable.index)

timetable_df["fifth_timestamp"] = timetable.apply(lambda x: x[0])
timetable_df["fourth_timestamp"] = timetable.apply(lambda x: x[1] if len(x) > 1 else 0)
timetable_df["third_timestamp"] = timetable.apply(lambda x: x[2] if len(x) > 2 else 0)
timetable_df["second_timestamp"] = timetable.apply(lambda x: x[3] if len(x) > 3 else 0)
timetable_df["first_timestamp"] = timetable.apply(lambda x: x[4] if len(x) > 4 else 0)

timetable_df = timetable_df/8.64e+7

In [ ]:
train["time_diff"] = train.groupby("user_id")["timestamp"].diff().fillna(0).astype("float32")
train["ts_diff_shifted"] = train.groupby("user_id")["time_diff"].shift(1).replace(0, np.nan)
train["ts_diff_shifted"] = train.groupby("user_id")["ts_diff_shifted"].fillna(method="ffill").fillna(0)

In [ ]:
train["time_diff1"] = train.groupby("user_id")["timestamp"].diff(2).fillna(0).astype("float32")
train["ts_diff_shifted_2"] = train.groupby("user_id")["time_diff1"].shift(1).replace(0, np.nan)
train["ts_diff_shifted_2"] = train.groupby("user_id")["ts_diff_shifted_2"].fillna(method="ffill").fillna(0)

In [ ]:
ts_diff_shifted = train.groupby("user_id")["ts_diff_shifted"].last()
ts_diff_shifted2 = train.groupby("user_id")["ts_diff_shifted_2"].last()

In [ ]:
ts_diff_shifted = pd.DataFrame(ts_diff_shifted)
ts_diff_shifted.columns = ["ts_diff_shifted"]

ts_diff_shifted2 = pd.DataFrame(ts_diff_shifted2)
ts_diff_shifted = ts_diff_shifted.join(ts_diff_shifted2, how="left")
ts_diff_shifted.columns = ["ts_diff_shifted","ts_diff_shifted_2"]

In [ ]:
lecture_count = train.groupby("user_id")["content_type_id"].sum()
interactions = train.groupby("user_id")["content_type_id"].count()


base = pd.DataFrame(lecture_count)
base.columns = ["lecs_n"]

base = base.join(interactions, how="left")
base.columns = ["lecs_n", "interaction_n"]

In [ ]:
base = base.join(ts_diff_shifted)

In [ ]:
train["timestamp"] = train["timestamp"]/8.64e+7
train["time_diff"] = train.groupby("user_id")["timestamp"].diff().fillna(0).astype("float32")

In [ ]:
train["sessions"] = train["time_diff"] > 0.083
sessions = train.groupby("user_id")["sessions"].sum()

base = base.join(sessions)
base = base.rename(columns={"user_id":"session"})


In [ ]:
train["sessions"] = train.groupby("user_id")["sessions"].cumsum()

train["session_count"] =1
train["session_count"] = train.groupby(["user_id","sessions"])["session_count"].cumsum()

In [ ]:
session_count = train.groupby("user_id")[["session_count", "user_id"]].tail(1).set_index("user_id", drop=True)
base = base.join(session_count)

del train["session_count"]
del train["sessions"]

In [ ]:
train["mean_pause"] = ((train.time_diff > 0.083)*train.time_diff).replace(0,np.nan)
test = train[["mean_pause","user_id"]]
test = test.dropna()

base = base.join(test.groupby("user_id")["mean_pause"].sum())
base = base.rename(columns={"mean_pause":"sum_pauses"})
base.sum_pauses = base.sum_pauses.fillna(0)

In [ ]:
train = train[train["content_type_id"] == False]

In [ ]:
train.prior_question_had_explanation = train.prior_question_had_explanation.fillna(False).astype("bool")
train.prior_question_elapsed_time = train.prior_question_elapsed_time.fillna(0)

In [ ]:
base = base.join(train.groupby("user_id")["prior_question_had_explanation"].sum())
base = base.rename(columns={"prior_question_had_explanation":"had_exp"})

In [ ]:
train = train.merge(que_data[["part"]], how="left", right_index=True, left_on="content_id")
train.part = train.part.astype("int8")

In [ ]:
train["last_part"] = train.groupby("user_id")["part"].shift().fillna(-1).astype("int8")

In [ ]:
a = train.groupby("user_id")[["user_id","last_part"]].tail(1).set_index("user_id", drop=True)
base = base.join(pd.DataFrame(a, columns=["last_part"]))

In [ ]:
def f(x):
    
    parts = x.part.values.tolist()
    nb = x.answered_correctly.tolist()
    ret = [0]*7
    
    for i, k in enumerate(parts):
        ret[k-1] = nb[i]
        
    return ret

In [ ]:
a = pd.DataFrame(train.groupby(["user_id","part"])["answered_correctly"].count())

a["user_id"] = a.index.get_level_values(0)
a["part"] = a.index.get_level_values(1)

a.reset_index(drop=True, inplace=True)
base = base.join(pd.DataFrame(a.groupby("user_id").apply(f), columns=["part_count"]))

In [ ]:
a = pd.DataFrame(train.groupby(["user_id","part"])["answered_correctly"].sum())

a["user_id"] = a.index.get_level_values(0)
a["part"] = a.index.get_level_values(1)

a.reset_index(drop=True, inplace=True)
base = base.join(pd.DataFrame(a.groupby("user_id").apply(f), columns=["part_corr"]))

In [ ]:
tmp_train = pd.read_pickle("/home/dfs/data_2")

tmp_train = tmp_train[["k", "content_id"]]
que_2_k = tmp_train.groupby("content_id")["k"].first().to_dict()

del tmp_train

In [ ]:
que_2_k = pd.DataFrame(pd.Series(que_2_k), columns=["k"])
train = train.merge(que_2_k, how="left", left_on="content_id", right_index=True)

In [ ]:
def f(x):
    
    k = x.k.values.tolist()
    nb = x.answered_correctly.tolist()
    ret = [0]*20
    
    for i, j in enumerate(k):
        ret[j] = nb[i]
        
    return ret

In [ ]:
a = pd.DataFrame(train.groupby(["user_id", "k"])["answered_correctly"].count())

a["user_id"] = a.index.get_level_values(0)
a["k"] = a.index.get_level_values(1)

a.reset_index(drop=True, inplace=True)

k_count = pd.DataFrame(a.groupby("user_id").apply(f), columns=["k_count"])

In [ ]:
a = pd.DataFrame(train.groupby(["user_id", "k"])["answered_correctly"].sum())

a["user_id"] = a.index.get_level_values(0)
a["k"] = a.index.get_level_values(1)

a.reset_index(drop=True, inplace=True)

k_corr = pd.DataFrame(a.groupby("user_id").apply(f), columns=["k_corr"])

In [ ]:
base = base.join(k_corr).join(k_count)
del k_corr
del k_count

In [ ]:
prior_part_mean_dict = {1: 22166.159642501425,
 2: 18714.69673913695,
 3: 23620.317746179924,
 4: 23762.753651169547,
 5: 25094.620302855932,
 6: 32417.37918735745,
 7: 47444.16407400242}

In [ ]:
a = pd.DataFrame(train.groupby(["user_id","part"])["prior_question_elapsed_time"].sum())

a["user_id"] = a.index.get_level_values(0)
a["part"] = a.index.get_level_values(1)

a.reset_index(drop=True, inplace=True)

In [ ]:
def f(x):
    
    parts = x.part.values.tolist()
    nb = x.prior_question_elapsed_time.tolist()
    ret = [0]*7
    
    for i, k in enumerate(parts):
        ret[k-1] = nb[i]
        
    return ret

part_et = pd.DataFrame(a.groupby("user_id").apply(f), columns=["part_et"])

In [ ]:
a = pd.DataFrame(train.groupby(["user_id","part"])["prior_question_had_explanation"].sum())

a["user_id"] = a.index.get_level_values(0)
a["part"] = a.index.get_level_values(1)

a.reset_index(drop=True, inplace=True)


def f(x):
    
    parts = x.part.values.tolist()
    nb = x.prior_question_had_explanation.tolist()
    ret = [0]*7
    
    for i, k in enumerate(parts):
        ret[k-1] = nb[i]
        
    return ret

part_explan = pd.DataFrame(a.groupby("user_id").apply(f), columns=["part_explan"])

In [ ]:
a = train.groupby("user_id")[["user_id","prior_question_had_explanation"]].tail(5)

priors_5 = pd.DataFrame(a.groupby("user_id")["prior_question_had_explanation"].apply(lambda x: x.tolist()))
priors_5.columns = ["priors_5"]

base = base.join(part_et).join(part_explan).join(priors_5)

In [ ]:
last_ts = train[train.answered_correctly == True].groupby("user_id")["timestamp"].last()

last_ts = pd.DataFrame(last_ts)
last_ts.columns = ["recent_corr"]

In [ ]:
el_sum = train.groupby("user_id")["prior_question_elapsed_time"].sum()

In [ ]:
el_sum = pd.DataFrame(el_sum)
el_sum.columns = ["el_sum"]

base = base.join(el_sum).join(last_ts)

In [ ]:
a = train.groupby("content_id")["answered_correctly"].agg(["count","mean"])

easy_questions = a[(a["mean"] > 0.8)]
easy_questions["easy_question"] = True

del easy_questions["mean"]
del easy_questions["count"]


hard_questions = a[(a["count"] > 100) & (a["mean"] < 0.4)]
hard_questions["hard_question"] = True

del hard_questions["mean"]
del hard_questions["count"]

In [ ]:
hardest = train[train.content_id.isin(hard_questions.index)].groupby("user_id")["answered_correctly"].agg(["count","sum"])
hardest.columns = ["hard_ct", "hard_cr"]

In [ ]:
hardest = train[train.content_id.isin(hard_questions.index)].groupby("user_id")["answered_correctly"].agg(["count","sum"])
hardest.columns = ["hard_ct", "hard_cr"]
base = base.join(hardest)

base.hard_ct = base.hard_ct.fillna(0).astype("int16")
base.hard_cr = base.hard_cr.fillna(0).astype("int16")

del hardest

In [ ]:
easiest = train[train.content_id.isin(easy_questions.index)].groupby("user_id")["answered_correctly"].agg(["count","sum"])
easiest.columns = ["easy_ct", "easy_cr"]
base = base.join(easiest)

base.easy_ct = base.easy_ct.fillna(0).astype("int16")
base.easy_cr = base.easy_cr.fillna(0).astype("int16")

del easiest

In [ ]:
"""Dictionary containing accuracy, number of questions and number of correct questions and last lecture"""

summ = train.groupby("user_id")[["answered_correctly"]].sum()
mean = train.groupby("user_id")[["answered_correctly"]].mean()
count = train.groupby("user_id")[["answered_correctly"]].count()

last_lecs = train[["user_id", "content_id"]].groupby("user_id").tail(1)
last_lecs.index = last_lecs.user_id
del last_lecs["user_id"]



user_info = summ.join(mean, how="left", rsuffix="_mean").join(count, how="left", rsuffix="_count").join(last_lecs, how="left", rsuffix="_lecs").join(timetable_df, how="left")
user_info.columns = ["correct_count", "mean_acc", "count", "last_lec","first_timestamp","second_timestamp", "third_timestamp","fourth_timestamp", "fifth_timestamp"]
user_info["tmp"] = 0


user_info.last_lec = user_info.last_lec.fillna(0)

user_info.mean_acc = user_info.mean_acc.astype("float16")
user_info.correct_count = user_info.correct_count.astype("int32")

In [ ]:
user_info = user_info.join(base)
user_info = user_info.to_dict("index")

In [ ]:
containers_mean = train.groupby("task_container_id")["answered_correctly"].mean()
containers_mean = containers_mean.to_dict()

In [ ]:
hard_questions = hard_questions["hard_question"].to_dict()
easy_questions = easy_questions["easy_question"].to_dict()

In [ ]:
repeated_que_count = train.groupby(["user_id", "content_id"])[["answered_correctly"]].count()

def f(x):
    return x.droplevel(0).answered_correctly.to_dict()

repeated_que_count = repeated_que_count.groupby(level=0).apply(f).to_dict()

In [ ]:
del train
train = dt.fread("/input/riiid-test-answer-prediction/train.csv").to_pandas()
train = train[train.content_type_id == False]

In [ ]:
heat = pd.read_pickle("/home/LGBM build and test/heat_pickle")

In [ ]:
nlargest_que =train.groupby("content_id")["user_id"].count().nlargest(300).index

#train = train[~train.index.isin(val_idx.row_id)]
train_value = train[["user_id","content_id","answered_correctly","task_container_id"]].values

In [ ]:
top_que_history = {}

from tqdm import tqdm

for i in tqdm(range(train_value.shape[0])):

    que_id = train_value[i, 1]
    user_id = train_value[i, 0]
    correct = train_value[i, 2]

    if que_id in nlargest_que:

        if correct == True: 

            if top_que_history.get(user_id, -1) == -1:
                top_que_history[user_id] = {}
            top_que_history[user_id][que_id] = True


In [ ]:
cols = []
for i in nlargest_que:
    cols.append("q_"+str(i))

In [ ]:
que_corr = {}

for i in cols:
    dct = heat[i].nlargest(6)[1:].to_dict()
    que_corr[int(i[2:])] = {int(j[2:]):dct[j] for j in dct}

In [ ]:
import pickle 


with open('groups', 'wb') as handle:
    pickle.dump(groups, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('que_2_k', 'wb') as handle:
    pickle.dump(que_2_k, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('repeated_que_count', 'wb') as handle:
    pickle.dump(repeated_que_count, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('hardest', 'wb') as handle:
    pickle.dump(hard_questions, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('easiest', 'wb') as handle:
    pickle.dump(easy_questions, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('user_info', 'wb') as handle:
    pickle.dump(user_info, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('watched_tags', 'wb') as handle:
    pickle.dump(watched_tags, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('containers_mean', 'wb') as handle:
    pickle.dump(containers_mean, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('top_que_history', 'wb') as handle:
    pickle.dump(top_que_history, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('que_corr', 'wb') as handle:
    pickle.dump(que_corr, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('nlargest_que.npy', 'wb') as f:
    np.save(f, nlargest_que)